In [1]:
# WATSON METHODS
#
#

import json
import math
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
#from endpoints.twitter.twitter import *



def get_tone_analyzer() -> ToneAnalyzerV3:
    with open("../credentials/watson_credentials.json", "r") as file:
        creds = json.load(file)
        authenticator = IAMAuthenticator(creds['API_KEY'])
        tone_analyzer = ToneAnalyzerV3(
            version=creds['VERSION'],
            authenticator=authenticator)
        tone_analyzer.set_service_url(creds['URL'])
    return tone_analyzer


# 
def get_sentiment(tweets: list) -> list:
    """tone analyzer api call from sentiment.json data"""
    tweet_str = tweets_to_string(tweets)
    ret = get_tone_analyzer().tone(
            tone_input=tweet_str,
            content_type='text/plain',
            sentences=True).get_result()
    #ret = format_sentiment(ret)
    return ret

def sort_tones(d1: dict) -> dict:
    #organizes output of tones to 4 tone categories (joy, sadness, anger, calm)
    output = {}
    temp_calm = 0
    temp_fear = 0
    temp_sad = 0
    temp_joy = 0
    temp_anger = 0
    
    counter = 0
    
    for i in d1:
        if i == 'confident' or i == 'tentative' or i == 'analytical':
            temp_calm += d1.get(i)
            counter += 1
        if i == 'fear':
            temp_fear += d1.get(i)
        if i == 'joy':
            temp_joy += d1.get(i)
        if i == 'sadness':
            temp_sad += d1.get(i)
        if i == 'anger':
            temp_anger += d1.get(i)
    #end of for loop
    
    #check if fear was measured
    if temp_fear > 0 and temp_calm > 0:
        if temp_calm - temp_fear > 0:
            temp_calm = temp_calm - temp_fear
            
    #check if calm was measured
    #find average of multiple calm tones
    if temp_calm > 0 and counter > 0:
        temp_calm = temp_calm / counter
    
    #double check calm score is > 0
    if temp_calm > 0:
        output.update( {'calm': temp_calm })
    #end of calm
    
    if temp_sad > 0:
        output.update( {'sadness': temp_sad} )
    if temp_joy > 0:
        output.update( {'joy': temp_joy} )
    if temp_anger > 0:
        output.update( {'anger': temp_anger} )
    
    return output

def format_sentiment(xd):
    tone_out = xd
    tone_out = tone_out.get('document_tone')
    tone_out = tone_out.get('tones')

    output = {}
    length = len(tone_out)
    i = 0
    x_1 = [] #scores
    x_2 = [] #names

    while(i < length):
        temp = tone_out[i]
        for key in temp:
            if key == 'score':
                x_1.append(temp.get(key))
            if key == 'tone_id':
                x_2.append(temp.get(key))
        i = i + 1
    #end of while loop
    
    length = len(x_1) 
    for i in range(length): 
        output.update( { x_2[i] : x_1[i] } )
    #end of for loop
    output = sort_tones(output)
    return output

def tweets_to_string(tweets: list):
    #converts a list of tweets to a single string without "RT "s and periods in between tweets.
    ret = ""
    for i in tweet_test: 
        rts = i[0]
        rts += i[1]
        rts += i[2]
        if(rts == "RT "):
            x = len(i)
            i = i[3:x]
        ret += i
        ret += ". "
    return ret

# END OF WATSON METHODS

In [ ]:
#TESTING WATSON METHODS
tones = get_sentiment(twitter.get_all_tweets('timdoozy'))
tones = format_sentiment(tones)
print(tones)

In [23]:
##   RS.PY

## Recommendation algorithm
import math

def get_spotify_playlist_data(username: str) -> list:
    scope = 'user-library-read user-top-read playlist-modify-public user-follow-read'
    redirect_uri = 'https://localhost:8000/callback'
    token = get_user_token(username, scope, redirect_uri) #spotify.get_user_token()
    if token:
        print('Success, got spotify token')
    else:
        print("Cant get token for ", username)
    sp = authenticate_spotify(token) #spotify.authenticate_spotify()
    trackList = get_all_songs(username, sp) #spotify.get_all_songs()
    trackList = get_tracks_with_features(trackList, sp) #spotify.get_tracks_with_features
    return trackList

#get_spotify_playlist_data('timdoozie')

def check_key(dict, key) -> bool:
    if key in dict.keys():
        return True
    else:
        return False
    
def adjust_songs(tone_in: dict, song_num: int):
    #Returns a new dictionary containing the amount of songs of each tone to produce in the playlist
    temp_calm = 0
    temp_joy = 0
    temp_anger = 0
    temp_sad = 0
    length = len(tone_in)
    #print(length)
    #print(tone_in)
    #print(tone_in['joy'])
    count = 0
    if 'joy' in tone_in.keys():
        temp_joy = tone_in['joy']
        count += 1
    if 'anger' in tone_in.keys():
        temp_anger = tone_in['anger']
        count += 1
    if 'sadness' in tone_in.keys():
        temp_sad = tone_in['sadness']
        count += 1
    if 'calm' in tone_in.keys():
        temp_calm = tone_in['calm']
        count += 1
    
    div = 1 / count
    sum1 = 0
    
    list_vals = [0, 0, 0, 0]
    #joy, anger, sad, calm
    
    #getsum1
    if temp_joy > 0:
        temp_joy *= song_num
        temp_joy *= div
        temp_joy = math.floor(temp_joy)
        #list_vals.append(temp_joy)
        list_vals[0] = temp_joy
        #tone_in['joy'] = temp_joy
        sum1 += temp_joy
    if temp_anger > 0:
        temp_anger *= song_num
        temp_anger *= div
        temp_anger = math.floor(temp_anger)
        #list_vals.append(temp_anger)
        list_vals[1] = temp_anger
        #tone_in['anger'] = temp_anger
        sum1 += temp_anger
    if temp_sad > 0:
        temp_sad *= song_num
        temp_sad *= div
        temp_sad = math.floor(temp_sad)
        #list_vals.append(temp_sad)
        list_vals[2] = temp_sad
        #tone_in['sad'] = temp_sad
        sum1 += temp_sad
    if temp_calm > 0:
        temp_calm *= song_num
        temp_calm *= div
        temp_calm = math.floor(temp_calm)
        #list_vals.append(temp_calm)
        list_vals[3] = temp_calm
        #tone_in['calm'] = temp_calm
        sum1 += temp_calm
    
    #show_vals(temp_sad, temp_joy, temp_anger, temp_calm)
    
    #for i in list_vals:
        #print(i)
    
    #list_vals[i] = 
    
    #sum1 vs song_num
    if sum1 < song_num:
        while sum1 < song_num:
            for i in range(len(list_vals)):
                if(list_vals[i] == 0):
                    i += 1
                else:
                    list_vals[i] = list_vals[i] + 1
                    sum1 += 1
    
    #for i in list_vals:
        #print(i)
        
    #list_val[joy, anger, sad, calm]
    ret_dic = tone_in.copy()
      
    #update the output dictionary with new values
    if 'joy' in ret_dic.keys():
        ret_dic['joy'] = list_vals[0]
    if 'anger' in ret_dic.keys():
        ret_dic['anger'] = list_vals[1]
    if 'sadness' in ret_dic.keys():
        ret_dic['sadness'] = list_vals[2]
    if 'calm' in ret_dic.keys():
        ret_dic['calm'] = list_vals[3]
        
    return ret_dic
    
    

    
    
    
def show_vals(sad, joy, anger, calm):
    print("Sad: %d" %sad)
    print("Joy: %d" %joy)
    print("Anger: %d" %anger)
    print("Calm: %d" %calm)
                

#adjust_songs(tones, 50)
#new_t = adjust_songs(tones, 50)

In [6]:
#### START OF REC_ALG
# 
# Twitter Data
tweet_test = get_all_tweets('timdoozy')
print(tweet_test)

['RT Quackity March has been a pretty insane year guys', 'RT matthwatson covid19 type beat', 'RT aanthonyy07 Day 6 of quarantine', 'Bbyzo and chill FQSwitchGiveaway', 'RT pulte Ill be giving 1000 TONIGHT to someone random who retweets this because lets all stay inside must be following me so I can', 'RT FoxSportsRadio Source Says Tom Brady is Signing With Tampa Bay Bucs Tomorrow', 'RT pokimanelol please stay home and play video games', 'RT neiltyson 314159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211706798214808651328230664709', 'RT CharmingVirago Please consider the people more at riskCoronavirusPandemic panicbuying', 'RT luka7doncic', 'Beware of the boomer remover Older people and people with severe chronic conditions should take special precauti', 'RT beeple tom hanks beating the shit out of coronavirus', 'RT TravisGafford Looking for work', 'RT LegionHoops Serge Ibaka kissing the ball after Rudy Gobert drains a free throw 2 days ago Yike

In [9]:
#sentiment data
tones = get_sentiment(tweet_test)
tones = format_sentiment(tones)
print(tones)

{'joy': 0.617353, 'anger': 0.542552}


In [28]:
#tone_song data (how many songs of each tone to include in final playlist)
tone_songs = adjust_songs(tones, 50)
print(tone_songs)

{'joy': 26, 'anger': 24}


In [24]:
#spotify music feature data
#THIS TAKES VERY LONG TO RUN
feature_data = get_spotify_playlist_data('timdoozie')
print(feature_data)

Success, got spotify token
...connecting to Spotify
Getting tracks from library...
...getting all tracks from playlists
TEST  no. of tracks:  50
ttv timdoozy  no. of tracks:  40
Chill Zone  no. of tracks:  40
Danger Zone  no. of tracks:  451
Getting tracks from user library..
...getting top and similar artists
...getting top tracks for each artist
[{'id': '2nr6iUHCilKUfyL9CicbQE', 'artist': 'Steely Dan', 'name': 'Do It Again', 'danceability': 0.683, 'instrumentalness': 4.16e-05, 'energy': 0.533, 'loudness': -10.253, 'speechiness': 0.0313, 'acousticness': 0.206, 'tempo': 124.571, 'liveness': 0.0525, 'valence': 0.964}, {'id': '7kv7zBjMtVf0eIJle2VZxn', 'artist': 'Glen Campbell', 'name': 'Southern Nights', 'danceability': 0.697, 'instrumentalness': 0.0232, 'energy': 0.813, 'loudness': -8.89, 'speechiness': 0.0315, 'acousticness': 0.0989, 'tempo': 95.301, 'liveness': 0.583, 'valence': 0.85}, {'id': '55GxhCTq6SY3tFTVh7z1nR', 'artist': 'King Harvest', 'name': 'Dancing In The Moonlight', 'danc

In [32]:
#given feature data lets be able to pick a song randomly given specific values to look for
df = pd.DataFrame.from_dict(feature_data)

In [33]:
df.head()

,id,artist,name,danceability,instrumentalness,energy,loudness,speechiness,acousticness,tempo,liveness,valence
0,2nr6iUHCilKUfyL9CicbQE,Steely Dan,Do It Again,0.683,0.000042,0.533,-10.253,0.0313,0.2060,124.571,0.0525,0.964
1,7kv7zBjMtVf0eIJle2VZxn,Glen Campbell,Southern Nights,0.697,0.023200,0.813,-8.890,0.0315,0.0989,95.301,0.5830,0.850
2,55GxhCTq6SY3tFTVh7z1nR,King Harvest,Dancing In The Moonlight,0.653,0.000000,0.719,-5.262,0.0363,0.6840,136.153,0.4080,0.682
3,3XF5xLJHOQQRbWya6hBp7d,Billie Eilish,you should see me in a crown,0.678,0.219000,0.533,-10.485,0.1860,0.4620,150.455,0.1390,0.323
4,3QzAOrNlsabgbMwlZt7TAY,Mustard,Ballin' (with Roddy Ricch),0.809,0.000000,0.545,-6.678,0.0735,0.3920,97.008,0.1510,0.244


In [34]:
df.describe()

,danceability,instrumentalness,energy,loudness,speechiness,acousticness,tempo,liveness,valence
count,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000
mean,0.627863,0.087170,0.619743,-7.368840,0.122987,0.247418,120.975245,0.199834,0.457763
std,0.154408,0.232442,0.204673,3.478104,0.111597,0.293152,31.708236,0.162114,0.224194
min,0.135000,0.000000,0.046600,-23.016000,0.026300,0.000012,47.811000,0.021000,0.033700
25%,0.518000,0.000000,0.498000,-8.901500,0.043350,0.014650,93.984000,0.099850,0.279000
50%,0.635000,0.000019,0.627000,-6.798000,0.073800,0.118000,120.113000,0.129000,0.447000
75%,0.730500,0.003230,0.768500,-4.990500,0.164500,0.414500,142.715000,0.240500,0.619500
max,0.935000,0.955000,0.997000,-1.270000,0.568000,0.986000,199.915000,0.982000,0.975000


In [57]:
#joy_tracks
joy_tracks = df.copy()
joy_tracks = joy_tracks.drop(['speechiness','instrumentalness','liveness','acousticness','name','artist','loudness','tempo','danceability'], axis=1)
joy_tracks = joy_tracks[joy_tracks['energy'] > 0.5]
joy_tracks = joy_tracks[joy_tracks['valence'] > 0.5]
joy_tracks = joy_tracks.sample(frac=1).reset_index(drop=True)
joy_tracks.head()

,id,energy,valence
0,777NOXupZkFdg8zlPMoVqc,0.619,0.740
1,4xlpJ99yL9xYQtzG6c3hwk,0.691,0.870
2,1jzIJcHCXneHw7ojC6LXiF,0.612,0.684
3,4S4Mfvv03M1cHgIOJcbUCL,0.647,0.670
4,6Z8R6UsFuGXGtiIxiD8ISb,0.819,0.766


In [59]:
#anger_tracks
anger_tracks = df.copy()
anger_tracks = anger_tracks.drop(['speechiness','instrumentalness','liveness','acousticness','name','artist','loudness','tempo','danceability'], axis=1)
anger_tracks = anger_tracks[anger_tracks['energy'] > 0.5]
anger_tracks = anger_tracks[anger_tracks['valence'] < 0.5]
anger_tracks = anger_tracks.sample(frac=1).reset_index(drop=True)
anger_tracks.head()

,id,energy,valence
0,3e0yTP5trHBBVvV32jwXqF,0.844,0.340
1,3Z25k4ZF6QENy2d9YatsM5,0.515,0.420
2,3NkJNL3WqO1Lqc3uNDxvCN,0.721,0.317
3,0AOvNRgl0SMfOibWA5bP8o,0.531,0.344
4,20hFFeL5QykY9VWeV5RayW,0.993,0.279


In [60]:
#sad_tracks
sad_tracks = df.copy()
sad_tracks = sad_tracks.drop(['speechiness','instrumentalness','liveness','acousticness','name','artist','loudness','tempo','danceability'], axis=1)
sad_tracks = sad_tracks[sad_tracks['energy'] < 0.5]
sad_tracks = sad_tracks[sad_tracks['valence'] < 0.5]
sad_tracks = sad_tracks.sample(frac=1).reset_index(drop=True)
sad_tracks.head()

,id,energy,valence
0,2MF4HtZHBoUliOi9nOAbS0,0.370,0.309
1,6DCZcSspjsKoFjzjrWoCdn,0.449,0.357
2,2INhKpUdzh7v0j041gZNsz,0.458,0.232
3,72794Eag03xdy7TO0KNuid,0.120,0.121
4,2VlLbhGLVJgdOW7kKdWWFc,0.307,0.157


In [62]:
#calm_tracks
calm_tracks = df.copy()
calm_tracks = calm_tracks.drop(['speechiness','instrumentalness','liveness','acousticness','name','artist','loudness','tempo','danceability'], axis=1)
calm_tracks = calm_tracks[calm_tracks['energy'] < 0.5]
calm_tracks = calm_tracks[calm_tracks['valence'] > 0.5]
calm_tracks = calm_tracks.sample(frac=1).reset_index(drop=True)
calm_tracks.head()

,id,energy,valence
0,2ZltjIqztEpZtafc8w0I9t,0.448,0.636
1,0chpIFSjgfd1CPrAxM7PWp,0.498,0.508
2,0F284njhROHN2imsXELE2D,0.275,0.508
3,6eGMwVVABqVTe9bWRIm498,0.464,0.771
4,0b9oOr2ZgvyQu88wzixux9,0.463,0.549


In [72]:
tttt = calm_tracks.at[0, 'id']
print(tttt)


2ZltjIqztEpZtafc8w0I9t


In [78]:
xd = calm_tracks.head(3)
xd.head()

,id,energy,valence
0,2ZltjIqztEpZtafc8w0I9t,0.448,0.636
1,0chpIFSjgfd1CPrAxM7PWp,0.498,0.508
2,0F284njhROHN2imsXELE2D,0.275,0.508


In [79]:
xd.drop(0)

,id,energy,valence
1,0chpIFSjgfd1CPrAxM7PWp,0.498,0.508
2,0F284njhROHN2imsXELE2D,0.275,0.508


In [58]:
joy_tracks = joy_tracks.sample(frac=1).reset_index(drop=True)
joy_tracks.head()


#joy_tracks.eval('energy > 0.5')
#df3 = result[result['Value'] > 10]  


#indexNames = joy_tracks['energy'] < 0.5.index
#indexNames2 = joy_tracks[ dfObj['valence'] < 0.5 ].index
# Delete these row indexes from dataFrame
#dfObj.drop(indexNames , inplace=True)


,id,energy,valence
0,5yY9lUy8nbvjM1Uyo1Uqoc,0.609,0.508
1,089keBWJUOsI9wUeVr1uIr,0.896,0.620
2,6SpLc7EXZIPpy0sVko0aoU,0.906,0.731
3,1nXZnTALNXiPlvXotqHm66,0.689,0.505
4,4MzGM692jKGkNSQ7qzeTaJ,0.674,0.743


In [30]:
#check what tones have been calculated
tone_scores = [0, 0, 0, 0]
songs_per_tone = [0, 0, 0, 0]
            #joy,anger,sadness,calm

#fill tone_scores
if 'joy' in tones.keys():
    tone_scores[0] = tones['joy']
if 'anger' in tones.keys():
    tone_scores[1] = tones['anger']
if 'sadness' in tones.keys():
    tone_scores[2] = tones['sadness']
if 'calm' in tones.keys():
    tone_scores[3] = tones['calm']

#fill tone_songs
if 'joy' in tone_songs.keys():
    songs_per_tone[0] = tone_songs['joy']
if 'anger' in tone_songs.keys():
    songs_per_tone[1] = tone_songs['anger']
if 'sadness' in tone_songs.keys():
    songs_per_tone[2] = tone_songs['sadness']
if 'calm' in tone_songs.keys():
    songs_per_tone[3] = tone_songs['calm']

for i in tone_songs:
    print(i)

for i in tone_scores:
    print(i)

print(len(tone_songs))

joy
anger
0.617353
0.542552
0
0
2


In [ ]:
playlist_tracks = []
count1 = 50
while(count1 > 0):
    for i in range(len(tone_scores)):
        if songs_per_tone[i] = 0:
            i += 1
        elif songs_per_tone[i] > 0 and i == 0:
            # joy song to be added to playlist
            temp_song = joy_tracks.at[0, 'id']
            # drop the song from the songlist
            joy_tracks.drop(0)
            
            # Add the retrieved song to the list of tracks
            playlist_tracks.append(temp_song)
            # Subtract after adding song to playlist output
            songs_per_tone[i] -= 1
            
        elif songs_per_tone[i] > 0 and i == 1:
            # anger song to be added to playlist
            temp_song = anger_tracks.at[0, 'id']
            # drop the song from the songlist
            anger_tracks.drop(0)
            # Shuffle anger_tracks
            anger_tracks = anger_tracks.sample(frac=1).reset_index(drop=True)
            # Add the retrieved song to the list of tracks
            playlist_tracks.append(temp_song)
            # Subtract after adding song to playlist output
            songs_per_tone[i] -= 1
            
        elif songs_per_tone[i] > 0 and i == 2:
            # sadness song to be added to playlist
            temp_song = sad_tracks.at[0, 'id']
            # drop the song from the songlist
            sad_tracks.drop(0)
            # Shuffle anger_tracks
            sad_tracks = sad_tracks.sample(frac=1).reset_index(drop=True)
            # Add the retrieved song to the list of tracks
            playlist_tracks.append(temp_song)
            # Subtract after adding song to playlist output
            songs_per_tone[i] -= 1
            
        elif songs_per_tone[i] > 0 and i == 3:
            # calm song to be added to playlist
            temp_song = calm_tracks.at[0, 'id']
            # drop the song from the songlist
            calm_tracks.drop(0)
            # Shuffle anger_tracks
            calm_tracks = calm_tracks.sample(frac=1).reset_index(drop=True)
            # Add the retrieved song to the list of tracks
            playlist_tracks.append(temp_song)
            # Subtract after adding song to playlist output
            songs_per_tone[i] -= 1

#return playlist_tracks

In [2]:
# TWITTER METHODS 
#
#

import tweepy
import json
import re
# path = 'data/'


def get_twitter_api() -> tweepy.API:
    """authenticates twitter and uses auth info to create and return a tweepy API"""
    with open("../credentials/twitter_credentials.json", "r") as file:
        creds = json.load(file)
    auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
    auth.set_access_token(creds['ACCESS_KEY'], creds['ACCESS_SECRET'])
    return tweepy.API(auth)


def remove_url(txt: str) -> str:
    """remove urls from tweets"""
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())


def get_tweets_by_user(username: str, num_tweets: int = 50) -> list:
    """get user's 'num_tweets' most recent tweets"""
    api = get_twitter_api()
    tweets_data = api.user_timeline(screen_name=username)
    tweets = [tweet_data.text for tweet_data in tweets_data]
    return tweets[:num_tweets]


    #200 is maximum amount of tweets at a time
def get_all_tweets(username: str) -> list:
    api = get_twitter_api() # always need this when making twitter api calls!
    alltweets = api.user_timeline(screen_name = username, count=200)
    outtweets = [remove_url(tweet.text) for tweet in alltweets]
    return outtweets

# END OF TWITTER METHODS

In [4]:
# SPOTIFY METHODS
import spotipy
import random
import json
import pandas as pd


def get_user_token(username: str, scope: str, redirect_uri: str) -> str:
    """get token for specified user via credentials"""
    with open("../credentials/spotify_credentials.json", "r") as file:
        creds = json.load(file)
    return spotipy.util.prompt_for_user_token(username, scope, creds['CLIENT_ID'], creds['CLIENT_SECRET'],
                                              redirect_uri)


def authenticate_spotify(token: str) -> spotipy.Spotify:
    """authenticates Spotify account via the passed in token"""
    print('...connecting to Spotify')
    sp = spotipy.Spotify(auth=token)
    return sp


def get_track_features(track_id: str, sp: spotipy.Spotify):
    """get dict of track features"""
    if track_id is None:
        return None
    else:
        features = sp.audio_features([track_id])
    return features


def get_tracks_with_features(tracks: list, sp: spotipy.Spotify) -> list:
    """return list of dicts with track info and features"""
    tracks_with_features = []
    for track in tracks:
        #print(track)
        features = get_track_features(track['id'], sp)
        if features:
            f = features[0]
            tracks_with_features.append(dict(
                                            id=track['id'],
                                            artist=track['artist'],
                                            name=track['name'],
                                            danceability=f['danceability'],
                                            instrumentalness=f['instrumentalness'],
                                            energy=f['energy'],
                                            loudness=f['loudness'],
                                            speechiness=f['speechiness'],
                                            acousticness=f['acousticness'],
                                            tempo=f['tempo'],
                                            liveness=f['liveness'],
                                            valence=f['valence']
                                            ))
    return tracks_with_features


def get_all_tracks_from_playlists(username: str, sp: spotipy.Spotify) -> list:
    print('...getting all tracks from playlists')
    playlists = sp.user_playlists(username)
    trackList = []
    for playlist in playlists['items']:
        if playlist['owner']['id'] == username:
            print(playlist['name'], ' no. of tracks: ', playlist['tracks']['total'])
            results = sp.user_playlist(username, playlist['id'], fields="tracks,next")
            tracks = results['tracks']
            for i, item in enumerate(tracks['items']):
                track = item['track']
                trackList.append(dict(id=track['id'], artist=track['artists'][0]['name'], name=track['name']))
    return trackList


def get_top_artists(sp: spotipy.Spotify, amount: int = 20) -> list:
    """compiles list of user's top artists of length amount"""
    print('...getting top artists')
    artists_name = []
    artists_uri = []
    ranges = ['short_term', 'medium_term', 'long_term']
    for r in ranges:
        all_top_artist_data = sp.current_user_top_artists(limit=amount, time_range=r)
        top_artist_data = all_top_artist_data['items']
        for artist_data in top_artist_data:
            if artist_data['name'] not in artists_name:
                artists_name.append(artist_data['name'])
                artists_uri.append(artist_data['uri'])
    return artists_uri


def get_top_and_similar_artists(sp: spotipy.Spotify, amount: int = 20) -> list:
    """compiles a list of top and similar artists of length amount"""
    print('...getting top and similar artists')
    artists_name = []
    artists_uri = []
    ranges = ['short_term', 'medium_term', 'long_term']
    for r in ranges:
        all_top_artist_data = sp.current_user_top_artists(limit=amount, time_range=r)
        top_artist_data = all_top_artist_data['items']
        for artist_data in top_artist_data:
            if artist_data['name'] not in artists_name and len(artists_uri) < amount:
                artists_name.append(artist_data['name'])
                artists_uri.append(artist_data['uri'])
                similar_artists_data = sp.artist_related_artists(artist_id=artist_data['uri'])
                for index, similar_artist_data in enumerate(similar_artists_data['artists']):
                    if similar_artist_data['name'] not in artists_name and len(artists_uri) < amount:
                        artists_name.append(similar_artist_data['name'])
                        artists_uri.append(similar_artist_data['uri'])
                    if index == 2:
                        break
    return artists_uri


def get_artists_top_tracks(sp: spotipy.Spotify, artists_uri: list, amount: int = 50) -> list:
    """compiles unordered list of top tracks made by artists in artists_uri of length amount"""
    print('...getting top tracks for each artist')
    tracks = []
    for artist in artists_uri:
        all_top_tracks_data = sp.artist_top_tracks(artist)
        top_tracks_data = all_top_tracks_data['tracks']
        for track_data in top_tracks_data:
            tracks.append(dict(id=track_data['id'], artist=track_data['artists'][0]['name'], name=track_data['name']))
            #tracks.append(track_data)
    #random.shuffle(tracks)
    #tracks = tracks[0:amount]
    return tracks


def create_playlist(sp: spotipy.Spotify, tracks: list, playlist_name: str, amount: int = 0):
    """creates a playlist or tracks from tracks_uri on the users account of length amount"""
    print('...creating playlist')
    if amount == 0:
        amount = len(tracks)
    user_id = sp.current_user()["id"]
    playlist_id = sp.user_playlist_create(user_id, playlist_name)["id"]
    random.shuffle(tracks)
    tracks_uri = []
    for track in tracks[0:amount]:
        tracks_uri.append(track['uri'])
    sp.user_playlist_add_tracks(user_id, playlist_id, tracks_uri)
    print('playlist, {}, has been generated.'.format(playlist_name))


def write_to_csv(track_features):
    df = pd.DataFrame(track_features)
    df.drop_duplicates(subset=['name', 'artist'])
    print('Total tracks in data set', len(df))
    df.to_csv('mySongsDataSet.csv', index=False)

def show_tracks(tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print("   %d %32.32s %s" % (i, track['artists'][0]['name'],
            track['name']))

def get_recent_tracks(username: str, sp: spotipy.Spotify) -> list:
    print('...getting the recent tracks from a user')
    recent_tracks = []
    afterTime = time.time()
    print(afterTime)
    #afterTime = afterTime - 2629743  #one month ago
    afterTime = afterTime - 604800 #one week ago
    print(afterTime)
    recent_tracks = sp.current_user_recently_played(after=afterTime)
    return recent_tracks
    

def get_all_songs(username: str, sp: spotipy.Spotify) -> list:
    #Method which gets songs from library, playlists, and top artists and similar artists into 1 dict
    print("Getting tracks from library...")
    trackList = [] #returning list
    #get all songs from playlists
    playlists = get_all_tracks_from_playlists(username, sp)
    #get all songs from library
    library = get_library(username, sp)
    #merge library and tracklist
    trackList = merge_dicts(playlists, library)
    #get all songs from top artists and similar artists
    preferences = get_artists_top_tracks(sp, get_top_and_similar_artists(sp))
    #merge lists again
    trackList = merge_dicts(preferences, trackList)
    return trackList

def merge_dicts(list1: list, list2: list) -> list:
    #append two dictionaries together
    for item in list1:
        if item not in list2:
            list2.append(item)
    return list2

def get_library(username: str, sp: spotipy.Spotify) -> list:
    print("Getting tracks from user library..")
    trackList = []
    library = sp.current_user_saved_tracks()
    for item in library['items']:
        track = item['track']
        trackList.append(dict(id=track['id'], artist=track['artists'][0]['name'], name=track['name']))
    return trackList



"""
# TO TEST FUNCTIONS

username = 'atamargo'
scope = 'user-library-read user-top-read playlist-modify-public user-follow-read'
redirect_uri = 'https://localhost:8000/callback'

token = get_user_token(username, scope, redirect_uri)

if token:
    sp = authenticate_spotify(token)
    results = get_artists_top_tracks(sp, get_top_and_similar_artists(sp))
    results = get_tracks_with_features(results, sp)
    print('\nTOP TRACKS WITH FEATURES\n')
    for result in results:
        print('{0} - {1}'.format(result['artist'], result['name']))
    create_playlist(sp, results, "TEST")
else:
    print("Can't get token for ", username)
"""
# END OF SPOTIFY METHODS

'\n# TO TEST FUNCTIONS\n\nusername = \'atamargo\'\nscope = \'user-library-read user-top-read playlist-modify-public user-follow-read\'\nredirect_uri = \'https://localhost:8000/callback\'\n\ntoken = get_user_token(username, scope, redirect_uri)\n\nif token:\n    sp = authenticate_spotify(token)\n    results = get_artists_top_tracks(sp, get_top_and_similar_artists(sp))\n    results = get_tracks_with_features(results, sp)\n    print(\'\nTOP TRACKS WITH FEATURES\n\')\n    for result in results:\n        print(\'{0} - {1}\'.format(result[\'artist\'], result[\'name\']))\n    create_playlist(sp, results, "TEST")\nelse:\n    print("Can\'t get token for ", username)\n'

In [ ]:
###
###
###
#get_recent_tracks
print(tones)

In [ ]:
# Testing methods
# 
# Twitter Test
tweet_test = get_all_tweets('timdoozy')
#tweet_test = get_tweets_by_user('timdoozy')
print(tweet_test)

In [ ]:
#print(tones)
def sort_tones(d1: dict) -> dict:
    output = {}
    temp_calm = 0
    temp_fear = 0
    temp_sad = 0
    temp_joy = 0
    temp_anger = 0
    for i in d1:
        if i == 'confident' or i == 'tentative' or i == 'analytical':
            temp_calm += d1.get(i)
        if i == 'fear':
            temp_fear += d1.get(i)
        if i == 'joy':
            temp_joy += d1.get(i)
        if i == 'sadness':
            temp_sad += d1.get(i)
        if i == 'anger':
            temp_anger += d1.get(i)
        #output.update( { i : d1.get(i) } )
    #end of for loop
    
    if temp_fear > 0 and temp_calm > 0:
        if temp_calm - temp_fear > 0:
            temp_calm = temp_calm - temp_fear
            output.update( {'calm': temp_calm} )
    #end of calm
    
    if temp_sad > 0:
        output.update( {'sadness': temp_sad} )
    if temp_joy > 0:
        output.update( {'joy': temp_joy} )
    if temp_anger > 0:
        output.update( {'anger': temp_anger} )
    
    return output
        
o = sort_tones(tones)
print(o)

In [ ]:
##SORT TONES
print(tones)
sorted_tones = {
    'joy': 0,
    'anger': 0,
    'sad': 0,
    'calm': 0
}
temp_calm = 0
temp_fear = 0
calm_counter = 0
for key,val in tones.items():
    if (key == 'joy') and (val>0):
        sorted_tones.update({'joy': val*100})
    if (key == 'anger') and (val>0):
        sorted_tones.update({'anger': val*100})
    if (key == 'sad') and (val>0):
        sorted_tones.update({'sad': val*100})
    if (key == 'tentative') and (val>0):
        temp_calm = temp_calm + val
        calm_counter = calm_counter + 1
    if (key == 'confident') and (val>0):
        temp_calm = temp_calm + val
        calm_counter = calm_counter + 1
    if (key == 'analytical') and (val>0):
        temp_calm = temp_calm + val
        calm_counter = calm_counter + 1
    if (key == 'fear') and (val>0):
        temp_fear = temp_fear + val
    print(key, "=>", val)

print(temp_calm)
print(temp_fear)
#subtract fear from calmness
if (temp_calm > 0):
    if (temp_fear > 0) and (temp_fear < temp_calm):
        temp_calm = temp_calm - temp_fear
#weigh calmness
if (temp_calm > 0):
    temp_calm = temp_calm / calm_counter
    
sorted_tones.update({'calm': temp_calm*100})

# END OF SORTED TONES
#return sorted_tones


In [ ]:
sorted_tones2 = {
    'joy': 0.6783
    'anger': 0.4302
    'sad': 0.0
    'calm': 0.0
}
song_num2 = 50
# given sorted tones and song_number input weigh out the songs based on tone scores

sorted_tones2.update({'joy': })

denom = 0
67.83%
43.02%












In [ ]:
joy = tones.fromkey('joy')
print(joy)
for key,val in tones.items():
    if (key == 'joy') and (val>0):
        sorted_tones.update({'joy': val})
    if (key == 'anger') and (val>0):
        sorted_tones.update({'anger': val})
    if (key == 'sad') and (val>0):
        sorted_tones.update({'sad': val})
    if (key == 'tentative') and (val>0):
        temp_calm = temp_calm + val
        calm_counter = calm_counter + 1
    if (key == 'confident') and (val>0):
        temp_calm = temp_calm + val
        calm_counter = calm_counter + 1
    if (key == 'analytical') and (val>0):
        temp_calm = temp_calm + val
        calm_counter = calm_counter + 1
    if (key == 'fear') and (val>0):
        temp_fear = temp_fear + val
    print(key, "=>", val)